In [1]:
import ee, eemont, geemap.eefolium as geemap

In [3]:
ee.Initialize()

In [ ]:
point = ee.Geometry.Point([-76.0269,2.92846])

In [ ]:
S2 = (ee.ImageCollection('COPERNICUS/S2_SR')
      .filterBounds(point)
      .sort('CLOUDY_PIXEL_PERCENTAGE')
      .first()
      .maskClouds()
      .scale()
      .index('NDSI'))

In [ ]:
NDSI = S2.select('NDSI')
N = S2.select('B8')
G = S2.select('B3')

In [ ]:
snowPixels = (NDSI > 0.4) & (N >= 0.1) & (G > 0.11)

In [ ]:
NDSI = NDSI.updateMask(snowPixels)

In [ ]:
dateOfInterest = ee.Date(S2.get('system:time_start'))

In [ ]:
MOD10A1 = (ee.ImageCollection('MODIS/006/MOD10A1')
           .filterBounds(point)
           .closest(dateOfInterest)
           .scale() # NEW! Note that the scale() method supports the MOD10A1 product!
           .first())

In [ ]:
NDSI_MODIS = MOD10A1.select('NDSI')
NDSI_MODIS = NDSI_MODIS.updateMask(NDSI_MODIS > 0.4)

In [ ]:
visNDSI = {
    'min':0.4,
    'max':1,
    'palette': ['black', '0dffff', '0524ff', 'ffffff']
}

In [ ]:
visRGB = {
    'min':0,
    'max':0.3,
    'bands':['B4', 'B3', 'B2']
}

In [ ]:
Map = geemap.Map()
Map.addLayer(S2,visRGB,'Sentinel-2 RGB')
Map.addLayer(NDSI_MODIS,visNDSI,'MODIS NDSI')
Map.addLayer(NDSI,visNDSI,'Sentinel-2 NDSI')
Map.centerObject(point,13)
Map.addLayerControl()
Map